## Introduction

PowerShell is a command shell, meaning you can open a 'shell' (a historical term for any program that allows human interaction with the OS) and send instructions to, or retrieve information from, the operating system in the form of PowerShell commands. It is also a scripting language, meaning these commands can be written into files and ran, encapsulated together as a rudiementary program (although, beyond basic scripts PowerShell is in fact a full programming language). 

An important conceptual difference between PowerShell and older unix-style command shells, which worked solely on text streams, is that PowerShell is _object_-first; the input and output of every command is an **object**.

Like objects in object-orientated programming, PowerShells objects have _members_, which are the data stored in the object and the functionality that can be used on the object. PowerShell extends this by providing the means to pass objects to commands. 

The below command `Get-Process` retrieves information about the process named 'code' (vscode) and constructs a PowerShell style object with that information. Passing it via the _pipeline_ (the '|' operator) to another command, `Get-Member`, it is possible to see the various member functions that can be used on this object, such as to kill the process, as well as the data members that store information about the process, such as the number of threads.

In [1]:
Get-Process code | Select-Object -First 1 | Get-Member Kill, Threads 


   TypeName: System.Diagnostics.Process

Name    MemberType Definition
----    ---------- ----------
Kill    Method     void Kill(), void Kill(bool entireProcessTree)
Threads Property   System.Diagnostics.ProcessThreadCollection Threads {get;}



## Commands
PowerShell commands can be either native binaries, such as _whoami.exe_ or _net.exe_, PowerShell _cmdlets_, which are PowerShell commands written in .NET (or sometimes in PowerShell itself), as well as PowerShell _scripts_, _functions_, or _modules_ (which are collections of any of the above that can be distrubted and imported as needed).

Cmdlets are a powerful feature of PowerShell, as it allows for commands to be written in a common language and against a common runtime.

In [2]:
Get-Command -Type Cmdlet Get-D*


CommandType     Name                                               Version    Source
-----------     ----                                               -------    ------
Cmdlet          Get-DAPolicyChange                                 2.0.0.0    NetSecurity
Cmdlet          Get-Date                                           7.0.0.0    Microsoft.PowerShell…
Cmdlet          Get-DeliveryOptimizationLog                        1.0.3.0    DeliveryOptimization
Cmdlet          Get-DeliveryOptimizationLogAnalysis                1.0.3.0    DeliveryOptimization



## Nouns and Verbs
PowerShell commands follow a verb-noun structure, such as `Get-Date` above. As in the english language, the verb is the action that the command will perform, and the noun is the object the action will be performed on. The `Get-Verb` command below will return information on every kind of verb PowerShell knows about, but for brevity only three are shown here.

In [3]:
Get-Verb Get, Set, Add


Verb AliasPrefix Group  Description
---- ----------- -----  -----------
Add  a           Common Adds a resource to a container, or attaches an item to another item
Get  g           Common Specifies an action that retrieves a resource
Set  s           Common Replaces data on an existing resource or creates a resource that contains …



## Drives
PowerShell uses the concept of _drives_ to represent the different parts of the operating system it can interact with, from a filesystem drive (inlcuding network drives), to the registry, to more abstract drives such as environmental variables. Each kind of drive is governed by a _provider_, which is a software layer that provides the means to interact with that particular operating system component.

In [4]:
Get-PSDrive -PSProvider FileSystem, Registry, Environment


Name           Used (GB)     Free (GB) Provider      Root                                CurrentLoc
                                                                                              ation
----           ---------     --------- --------      ----                                ----------
C                 142.46         95.24 FileSystem    C:\                                 …owerShell
Env                                    Environment                                                 
HKCU                                   Registry      HKEY_CURRENT_USER                             
HKLM                                   Registry      HKEY_LOCAL_MACHINE                            
Temp              142.46         95.24 FileSystem    C:\Users\alex\AppData\Local\Temp\             



## Working Location
Like other command shells, PowerShell uses the notion of the _working location_; usually this will be the current folder. However it is possible to be working from a location in a non-filesystem drive as well.

In [5]:
Get-Location


Path
----
C:\Users\alex\Projects\LearningPowerShell



In [6]:
Set-Location -Path HKLM:\SOFTWARE -PassThru



Path
----
HKLM:\SOFTWARE



If the above cell containing the 'Set-Location' cmdlet was ran, the below 'Get-Location' invocation should return the SOFTWARE registry key.

In [7]:
Get-Location


Path
----
HKLM:\SOFTWARE



## Items
Items are objects within a specific PS drive. For example, items in a filesystem drive are files, while items in a registry drive are registry keys.

In [8]:
Get-Item * | Select-Object -Property Name


Name
----
HKEY_LOCAL_MACHINE\SOFTWARE\7-Zip
HKEY_LOCAL_MACHINE\SOFTWARE\Adobe
HKEY_LOCAL_MACHINE\SOFTWARE\Classes
HKEY_LOCAL_MACHINE\SOFTWARE\Clients
HKEY_LOCAL_MACHINE\SOFTWARE\CVSM
HKEY_LOCAL_MACHINE\SOFTWARE\DefaultUserEnvironment
HKEY_LOCAL_MACHINE\SOFTWARE\dotnet
HKEY_LOCAL_MACHINE\SOFTWARE\Google
HKEY_LOCAL_MACHINE\SOFTWARE\Intel
HKEY_LOCAL_MACHINE\SOFTWARE\Khronos
HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft
HKEY_LOCAL_MACHINE\SOFTWARE\Mozilla
HKEY_LOCAL_MACHINE\SOFTWARE\mozilla.org
HKEY_LOCAL_MACHINE\SOFTWARE\ODBC
HKEY_LOCAL_MACHINE\SOFTWARE\OEM
HKEY_LOCAL_MACHINE\SOFTWARE\OpenSSH
HKEY_LOCAL_MACHINE\SOFTWARE\Oracle
HKEY_LOCAL_MACHINE\SOFTWARE\Partner
HKEY_LOCAL_MACHINE\SOFTWARE\Policies
HKEY_LOCAL_MACHINE\SOFTWARE\RegisteredApplications
HKEY_LOCAL_MACHINE\SOFTWARE\Setup
HKEY_LOCAL_MACHINE\SOFTWARE\WOW6432Node



## Streams and Redirection
'Streams' in PowerShell are output channels for different kinds of messages, such as errors or debug information, or console output.  
For example, the most commonly used stream is the **Success** stream, which is the stream which passes objects along the pipeline, and is also connected to the **stdout** of terminal applications for displaying text data to a terminal. 

In [22]:
$s = "The Write-Output cmdlet writes to the Success stream, which is the stream that handles output from the successful execution of commands"
Write-Output $s

The Write-Output cmdlet writes to the Success stream, which is the stream that handles output from the successful execution of commands


Streams are identified by number, and using the stream identifiers and the redirect operator `>` (similar to bash on linux), it is possible to re-route streams, outputting errors to the success stream for example, or all streams to a file.

In [41]:
Write-Output "Success: 1"
Write-Error "2" # this will also generate an error message of its own
Write-Warning "3"
Write-Verbose -Verbose "4"


Success: 1
Write-Error: 2
VERBOSE: 4


Error: Command failed: SubmitCode: Write-Output "Success: 1" ...